In [1]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
from torch.nn.init import kaiming_uniform_
from torch.distributions import Normal

In [2]:
torch.normal(mean=torch.arange(1., 6.), std=torch.arange(1., 6.))

tensor([ 2.6046, -1.6558,  1.5283,  6.9046,  5.3011])

In [3]:
import numpy as np
import gym
from tqdm import tqdm
import random as rand
from itertools import count

In [4]:
class ReplayMemory():
    def __init__(self,capacity):   
        self.capacity = capacity
        self.memory = []
        self.push_count = 0
        
    def push(self, experience):
        if len(self.memory) < self.capacity:
            self.memory.append(experience)
        else:
            self.memory[self.push_count%self.capacity] = experience
        self.push_count+=1
    
    def sample(self, batch_size):
        return rand.sample(self.memory,batch_size)
    
    def can_provide_sample(self, batch_size):
        return len(self.memory)>=batch_size
    
    def update_td_error(self, sampled_experiences):
        for sampled_idx,sampled_exp in enumerate(sampled_experiences):
            for mem_idx, mem_exp in enumerate(self.memory):
                if mem_exp.timestep == sampled_exp.timestep:
                    self.memory[mem_idx] = sampled_exp #update memory
                    break
        
    def get_memory_values(self):
        return self.memory    

In [5]:
def extract_tensors(experiences):
    #print(".....................................................")
    #print(experiences)
    batch = Xp(*zip(*experiences))
    state = np.stack(batch.state) #stack
    action = np.stack(batch.action)
    next_state = np.stack(batch.next_state)
    reward = np.stack(batch.reward)
    done = np.stack(batch.done)
    abs_td_error = np.stack(batch.abs_td_error)
    timestep = np.stack(batch.timestep)
    return state,action,next_state,reward,done,abs_td_error,timestep

In [6]:
def rebuild_experiences(state, action, next_state, reward, done, abs_error, timestep):
    exp_list = []
    for idx_ in range(len(state)):
        exp_list.append(\
                        Xp(state[idx_], action[idx_], next_state[idx_], reward[idx_],\
                           done[idx_], abs_error[idx_], timestep[idx_]))
    return exp_list

In [7]:
from collections import namedtuple
Xp = namedtuple('Experience',
                        ('state', 'action', 'next_state', 'reward', 'done', 'abs_td_error','timestep'))
Xp_points = Xp(5,6,7,8,9,10,11)
Xp_points

Experience(state=5, action=6, next_state=7, reward=8, done=9, abs_td_error=10, timestep=11)

In [8]:
def prioritize_samples(experience_samples, alpha, beta):
    state,action,next_state,reward,done,abs_td_error,timesteps = extract_tensors(experience_samples)
    #rank based
    #('state', 'action', 'next_state', 'reward', 'done', 'abs_td_error','timestep')
    abs_td_error  = np.expand_dims(abs_td_error, axis=1)
    abs_td_error = torch.tensor(abs_td_error)
    abs_td_error, indices_ = abs_td_error.sort(0, descending=True)#big to small
    indices = np.arange(1, len(abs_td_error)+1)
    priorities = 1.0/indices
    priorities = priorities**alpha#scale by alpha
    priorities = np.expand_dims(priorities, axis=1)
    probabilities = priorities/np.sum(priorities, axis=0)#sums up to 1(or 0.999999)
    assert np.isclose(probabilities.sum(), 1.0)#ensures probs add up to 1
    
    number_of_samples  = len(probabilities)
    weight_importance_ = number_of_samples*probabilities
    weight_importance_ = weight_importance_**-beta
    weight_importance_max = np.max(weight_importance_)
    weight_importance_scaled = weight_importance_/weight_importance_max
    return weight_importance_scaled, indices_ #return weight important samples, return indices for re_arranging sampled experiences

In [9]:
class linearApproximator_FCGSAP(nn.Module):
    def __init__(self,state_shape,outputs,hidden_dims=(32,32), log_entropy_lr =0.0001,\
                log_std_dev_min=-20, log_std_dev_max= 2, epsilon = 1e-6):
        super(linearApproximator_FCGSAP, self).__init__()
        self.input_size = state_shape
        self.out = outputs
        self.log_std_dev_min = log_std_dev_min
        self.log_std_dev_max = log_std_dev_max
        self.epsilon = epsilon
        self.device = torch.device("cuda" if torch.cuda.is_available()\
                                   else "cpu")
        
        self.fc1  = nn.Linear(self.input_size,hidden_dims[0])
        self.hidden_layers = nn.ModuleList()
        for i in range(len(hidden_dims)-1):
            hidden_layer = nn.Linear(\
                                hidden_dims[i], hidden_dims[i+1])
            self.hidden_layers.append(hidden_layer)
        
        self.output_layer_distribution  = nn.Linear(hidden_dims[-1],self.out)
        self.output_layer_mean = nn.Linear(hidden_dims[-1],self.out)
        
        self.target_entropy = -float(self.out)
        #self.target_entropy = self.target_entropy.float() 
        #according to the eq, log alpha is a learnable parameter
        self.log_alpha = torch.zeros(1,\
                                     requires_grad=True,\
                                     device = self.device)
        self.log_alpha_optimizer = torch.optim.Adam([self.log_alpha],\
                                                    lr=log_entropy_lr)
                                     
        self.to(self.device)
        
    def forward(self, state_shape):
        if not isinstance(state_shape, torch.Tensor):
            state_shape = torch.tensor(state_shape, dtype=torch.float32)
        state_shape = state_shape.to(self.device)
        x = self.fc1(state_shape)
        x = F.relu(x)
        
        for hidden_layer in self.hidden_layers:
            x = F.relu(hidden_layer(x))
        
        distribution = self.output_layer_distribution(x)#logits, preferences of actions
        mean   = self.output_layer_mean(x)
        distribution = torch.clamp(distribution, self.log_std_dev_min, self.log_std_dev_max)
        return mean, distribution
        
    def full_pass(self, state):
        mean, distribution = self.forward(state)
        pi_s = Normal(mean, distribution.exp())
        pre_sampled_actions = pi_s.rsample()
        sampled_actions = torch.tanh(pre_sampled_actions)#scale actions between -1 and 1
        #we also rescale our logprobs to match action space
        log_probs = pi_s.log_prob(pre_sampled_actions) - \
                                    torch.log((1 - sampled_actions.pow(2)).clamp(0,1) + \
                                                self.epsilon)
        log_probs = log_probs.sum(dim=1, keepdim=True)
        return sampled_actions, log_probs, mean

In [10]:
class linearApproximator_FCQV(nn.Module):#Q value of state action pair
    def __init__(self,state_shape,action_outputs_size,hidden_dims=(32,32)):
        super(linearApproximator_FCQV, self).__init__()
        self.input_size = state_shape
        self.action_outputs_size = action_outputs_size
        self.device = torch.device("cuda" if torch.cuda.is_available()\
                                   else "cpu")
        
        self.fc1  = nn.Linear(self.input_size,hidden_dims[0])
        self.hidden_layers = nn.ModuleList()
        for i in range(len(hidden_dims)-1):
            hidden_input_layer = hidden_dims[i]
            if i == 0:
                hidden_input_layer += self.action_outputs_size #increased to account for size/number of actions
            hidden_layer = nn.Linear(\
                                hidden_input_layer, hidden_dims[i+1])
            self.hidden_layers.append(hidden_layer)
        
        self.output_layer  = nn.Linear(hidden_dims[-1],1)
        self.to(self.device)
        
    def forward(self, state_shape, action_shape):
        if not isinstance(state_shape, torch.Tensor):
            state_shape = torch.tensor(state_shape, dtype=torch.float32).to(self.device)
        if not isinstance(action_shape, torch.Tensor):
            action_shape = torch.tensor(action_shape, dtype=torch.float32).to(self.device)
                    
        x = self.fc1(state_shape)
        x = F.relu(x)
        
        for idx, hidden_layer in enumerate(self.hidden_layers):
            if idx == 0:
                x = torch.cat((x, action_shape), dim=1)
            x = F.relu(hidden_layer(x))
        
        q_value = self.output_layer(x)#logits, preferences of actions
        return q_value

In [11]:
def update_networks(online_q_network_a, online_q_network_b,\
                    offline_q_network_a, offline_q_network_b, tau):
        
    for target_weights, online_weights in zip(offline_q_network_a.parameters(), online_q_network_a.parameters()):
        target_weight_update = (1.0 - tau)*target_weights.data
        online_weight_update = tau*online_weights.data
        sum_up = target_weight_update + online_weight_update
        target_weights.data.copy_(sum_up)
        
    for target_weights, online_weights in zip(offline_q_network_b.parameters(), online_q_network_b.parameters()):
        target_weight_update = (1.0 - tau)*target_weights.data
        online_weight_update = tau*online_weights.data
        sum_up = target_weight_update + online_weight_update
        target_weights.data.copy_(sum_up)

    return offline_q_network_a, offline_q_network_b

In [12]:
def update_online_model(experience_samples,\
                        online_policy_network, online_q_network_a, online_q_network_b,\
                        online_policy_optimizer, online_q_optimizer_a, online_q_optimizer_b,\
                        offline_q_network_a, offline_q_network_b,\
                        gamma, weighted_importance, indices):
    
    states, actions, next_states, rewards, done, td_errors, timesteps = extract_tensors(experience_samples)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    #rearrange first
    indices = np.array(indices)#have to be same size as batch size(already taken care of)
    indices = np.squeeze(indices)
    states = states[indices,:]
    states = np.squeeze(states)
    actions = actions[indices]
    next_states = next_states[indices,:]
    next_states = np.squeeze(next_states)
    rewards = rewards[indices]
    done = done[indices]
    td_errors = td_errors[indices]
    timesteps = timesteps[indices]    
    
    states = torch.tensor(states).float().to(device)
    actions = torch.tensor(actions)
    actions = actions.float().to(device)
    next_states=torch.tensor(next_states).float().to(device)
    rewards = torch.tensor(rewards).float().to(device)
    rewards = rewards.unsqueeze(1)
    done = torch.tensor(done).float().to(device)
    done = done.unsqueeze(1)
    weighted_importance = torch.tensor(weighted_importance).float().to(device)
    
    
    current_actions,log_pi, _ = online_policy_network.full_pass(states)
    target_alpha = (log_pi +\
                    online_policy_network.target_entropy).detach()
    target_alpha_loss = -(online_policy_network.log_alpha *\
                         target_alpha).mean()
    online_policy_network.log_alpha_optimizer.zero_grad()
    target_alpha_loss.backward()
    online_policy_network.log_alpha_optimizer.step()
    optimized_alpha = online_policy_network.log_alpha.exp()
    
    
    max_q_sa_online_a = online_q_network_a(states, current_actions.detach())
    max_q_sa_online_b = online_q_network_b(states, current_actions.detach())
    max_q_sa_online_a*=weighted_importance
    max_q_sa_online_b*=weighted_importance
    max_q_online__ = torch.min(max_q_sa_online_a, max_q_sa_online_b)
    #max_q_online__*=(1 - done)
    #print("max_q_online__", max_q_online__)
    predicted_online_action_policy,\
                log_pi_ns, _ = online_policy_network.full_pass(next_states)

    policy_loss = (-optimized_alpha.detach() *(max_q_online__.detach() - log_pi)).mean()#policy loss
    
    max_q_sa_offline_a = offline_q_network_a(next_states, predicted_online_action_policy)
    max_q_sa_offline_b = offline_q_network_b(next_states, predicted_online_action_policy)
    max_q_sa_offline = torch.min(max_q_sa_offline_a, max_q_sa_offline_b)
    TWIN_target = max_q_sa_offline
    TWIN_target*=weighted_importance
    
    TWIN_target = TWIN_target.detach()
    TWIN_target *=(1 - done)
    TWIN_target = TWIN_target - optimized_alpha * log_pi_ns
    TWIN_target = rewards + gamma*TWIN_target
    
    #abs_a = abs(TWIN_target.detach().cpu().numpy() - max_q_sa_online_a.detach().cpu().numpy())
    #abs_b = abs(TWIN_target.detach().cpu().numpy() - max_q_sa_online_b.detach().cpu().numpy())
    #ovr_abs_update = (abs_a + abs_b)/2 #we get the mean(not done)
    
    TWIN_target = TWIN_target.detach()
    loss_func = torch.nn.SmoothL1Loss()
    
    
    q_sa_online_a = online_q_network_a(states, actions)
    q_sa_online_b = online_q_network_b(states, actions)
    
    abs_a = abs(TWIN_target.detach().cpu().numpy() - max_q_sa_online_a.detach().cpu().numpy())
    abs_b = abs(TWIN_target.detach().cpu().numpy() - max_q_sa_online_b.detach().cpu().numpy())
    ovr_abs_update = (abs_a + abs_b)/2 #we get the mean(not done)
    
    
    
    q_online_value_loss_a = loss_func(q_sa_online_a,\
                                             TWIN_target)
    q_online_value_loss_b = loss_func(q_sa_online_b,\
                                             TWIN_target)
    online_q_optimizer_a.zero_grad()
    q_online_value_loss_a.backward()
    online_q_optimizer_a.step()
    online_q_optimizer_b.zero_grad()
    q_online_value_loss_b.backward()
    online_q_optimizer_b.step()
    
    #One of the difference between SAC and TD3 is SAC only uses one online policy
    #there is also no delay in policy updates in SAC
    #policy_loss = -(pre_optimized_alpha * log_pi - max_q_online__).mean()
    online_policy_optimizer.zero_grad()
    policy_loss.backward()
    online_policy_optimizer.step()
        
    states, actions, next_states, rewards, done, td_errors, timesteps = extract_tensors(experience_samples)
    experiences_rebuilded = rebuild_experiences(states, actions, next_states, rewards, done, ovr_abs_update, timesteps)
    return experiences_rebuilded

In [13]:
def query_error(online_policy_network, offline_q_network_a, offline_q_network_b,\
                online_q_network_a, online_q_network_b, state, action, next_state, reward, gamma):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    state = torch.tensor(state).float().to(device)
    state = state.unsqueeze(0)
    next_state = torch.tensor(next_state).float().to(device)
    next_state = next_state.unsqueeze(0)
    
    alpha = online_policy_network.log_alpha.exp()
    
    ns_actions,log_pi_ns, _ = online_policy_network.full_pass(next_state)
    q_target_next_states_action_a = offline_q_network_a(next_state,\
                                                    ns_actions.detach())
    q_target_next_states_action_b = offline_q_network_b(next_state,\
                                                    ns_actions.detach())
    
    
    TWIN_target = torch.min(q_target_next_states_action_a, q_target_next_states_action_b)
    TWIN_target = TWIN_target - alpha * log_pi_ns
    TWIN_target = reward + (gamma*TWIN_target.detach())
    
    
    current_action,_, _ = online_policy_network.full_pass(state)
    #print("state: ", state.shape)
    #print("action: ", action.shape)
    action = np.expand_dims(action, axis=0)
    #print("action: ", action.shape)
    q_online_state_action_val_a = online_q_network_a(state, action)
    q_online_state_action_val_b = online_q_network_b(state, action)
    
    abs_a = abs(TWIN_target - q_online_state_action_val_a)
    abs_b = abs(TWIN_target - q_online_state_action_val_b)
    abs_stack = (abs_a + abs_b)/2
    #print("abs querry error stacked: ", abs_stack.shape)
    ovr_abs_update = abs_stack
    #print("abs querry error mean: ", ovr_abs_update.shape)
    return ovr_abs_update.detach().cpu().numpy()

In [14]:
def freeze_model(model):
    for param in model.parameters():
        param.requires_grad = False
    return model

In [15]:
def select_action(state, online_policy_network):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    state = torch.tensor(state).float().to(device)
    state = state.unsqueeze(0)
    with torch.no_grad():
        actions,log_pi_, _ = online_policy_network.full_pass(state)
        actions = actions.cpu().detach()
        actions = actions.data.numpy().squeeze()
    return actions

In [21]:
def SAC_PER(env,
         gamma=0.99,
         alpha_pr=0.6,
         beta_pr=0.3,
         memory_size = 50000,
         tau = 0.1,
         offline_update = 1000,
         min_sample_size=200,
         batch_size = 64,
         n_ep=2000,
         max_steps = 100000
         ):
    
    
    observation_space = len(env.reset())
    action_space_high, action_space_low = env.action_space.high, env.action_space.low
    n_actions = len(action_space_high)
    online_policy_network = linearApproximator_FCGSAP(observation_space,n_actions,\
                                     hidden_dims=(128,64))
    online_q_network_a = linearApproximator_FCQV(observation_space,\
                                     n_actions,hidden_dims=(128,64))
    online_q_network_b = linearApproximator_FCQV(observation_space,\
                                     n_actions,hidden_dims=(128,64))
    
    offline_q_network_a = linearApproximator_FCQV(observation_space,\
                                     n_actions,hidden_dims=(128,64))
    offline_q_network_b = linearApproximator_FCQV(observation_space,\
                                     n_actions,hidden_dims=(128,64))
    

    offline_q_network_a.eval()
    offline_q_network_a = freeze_model(offline_q_network_a)
    offline_q_network_b.eval()
    offline_q_network_b = freeze_model(offline_q_network_b)
    
    online_policy_optimizer    = torch.optim.Adam(online_policy_network.parameters(),lr=0.0008)
    online_q_optimizer_a = torch.optim.Adam(online_q_network_a.parameters(),lr=0.0008)
    online_q_optimizer_b = torch.optim.Adam(online_q_network_b.parameters(),lr=0.0008)
    
    memory = ReplayMemory(memory_size)
    
    t_step = 0 #important
    reward_per_ep = []
    
    
    for e in tqdm(range(n_ep)):
        state = env.reset()
        reward_accumulated = 0
        
        while True:
            env.render()
            action = select_action(state, online_policy_network)
            if memory.can_provide_sample(min_sample_size):
                print(action)
            next_state, reward, done, info = env.step(action)
            td_error = query_error(online_policy_network, offline_q_network_a, offline_q_network_b,\
                online_q_network_a, online_q_network_b, state, action, next_state, reward, gamma)
            #print("td error: ", td_error.shape)
            td_error = np.squeeze(td_error, axis = 0)
            #print("td error: ", td_error.shape)
            reward_accumulated+=reward
            is_truncated = 'TimeLimit.truncated' in info and\
                                info['TimeLimit.truncated']
            is_failure = done and not is_truncated
           
            memory.push(Xp(state, action, next_state, reward, is_failure, td_error, t_step))
            state = next_state
            t_step+=1
            if memory.can_provide_sample(min_sample_size):
                experience_samples = memory.sample(batch_size)
                weighted_importance, indices = prioritize_samples(experience_samples, alpha_pr, beta_pr)
                rebuilded_exp = update_online_model(experience_samples,\
                        online_policy_network, online_q_network_a, online_q_network_b,\
                        online_policy_optimizer, online_q_optimizer_a, online_q_optimizer_b,\
                        offline_q_network_a, offline_q_network_b,\
                        gamma, weighted_importance, indices)
                memory.update_td_error(rebuilded_exp)
                
            if t_step%offline_update == 0:
                offline_q_network_a, offline_q_network_b = update_networks(online_q_network_a, online_q_network_b,\
                                                                    offline_q_network_a, offline_q_network_b, tau)
            if done == True:
                reward_per_ep.append(reward_accumulated)
                #print(reward_accumulated)
                break
            if t_step > max_steps:
                env.close()
                return reward_per_ep
    env.close()           
    return reward_per_ep

In [22]:
import gym
env = gym.make('BipedalWalker-v3')

In [23]:
env.close()

In [ ]:
rewards = SAC_PER(env)  

  0%|          | 2/2000 [00:01<17:35,  1.89it/s]

[-0.91166925  0.93979627  0.61329836  0.77086055]
[-0.20432463 -0.55195177 -0.5575151   0.02042458]
[-0.6556064   0.04860764  0.1192078   0.2332958 ]
[-0.6746763  -0.59897494 -0.55672646 -0.02301397]
[-0.9088265  -0.6487699  -0.2491401   0.97005314]
[-0.3964877   0.24614315  0.6312369   0.916448  ]
[ 0.3859649   0.23407508 -0.3061111   0.30990085]
[ 0.38699603  0.92000836  0.66908383 -0.27561423]
[ 0.6929807  -0.8136302  -0.2627812  -0.67524666]
[-0.42197585 -0.46928757 -0.14425729  0.8553563 ]
[-0.6435398   0.9486105  -0.5290562  -0.56782293]
[ 0.43598565  0.8003263  -0.83929884 -0.06605521]
[-0.5341318  -0.50468004 -0.99684066 -0.72267795]
[ 0.17672722 -0.1936523   0.90643644  0.9746543 ]
[ 0.46122405 -0.39175123  0.6966756  -0.5401637 ]
[ 0.7864884   0.8539301  -0.63390136  0.80754167]
[ 0.5646744 -0.965259   0.5555403  0.7077081]
[-0.3845712   0.75924957 -0.7033428   0.42812592]
[-0.3039445  -0.7051641  -0.05400967 -0.760298  ]
[-0.975991   0.0574446  0.7207432 -0.8646346]
[-0.2640

[ 0.39093345  0.45821247  0.26224166 -0.2752669 ]
[-0.8583696   0.17472805  0.76655555 -0.7686304 ]
[ 0.44114172 -0.81227654  0.83190453 -0.87917936]
[-0.58524567  0.25148252 -0.01041776  0.24781227]
[-0.3228092  -0.35129845  0.1412118   0.1808319 ]
[-0.641502   -0.08689647 -0.14849858 -0.5469134 ]
[ 0.22441703  0.24619113 -0.9519896  -0.9006433 ]
[-0.17518628  0.34878543  0.8340902   0.74889493]
[-0.36521104  0.8438494   0.7987218  -0.49486864]
[-0.48321578 -0.08072473  0.256328    0.77715814]
[-0.29321218 -0.5189447  -0.01753221  0.35817352]
[ 0.58172655  0.62542117 -0.80734664 -0.9078607 ]
[ 0.8525702  -0.47895387 -0.39903435 -0.64495814]
[ 0.98295635 -0.42522275 -0.61130595 -0.5719111 ]
[-0.11138701 -0.67985916  0.32145444  0.6336141 ]
[-0.7273824   0.7502688   0.03182378 -0.05845452]
[-0.8537618  -0.70404875  0.88679713 -0.4366827 ]
[ 0.941065   -0.2987885   0.7796695   0.41307962]
[0.59284604 0.31322655 0.725133   0.46856853]
[-0.09242254  0.6679096   0.85991347 -0.11825197]
[ 0.

[ 0.62159187 -0.35574302 -0.23401822 -0.67418766]
[ 0.15612654  0.9601529   0.82314205 -0.8357015 ]
[-0.40805328 -0.01029735 -0.48916733 -0.7460004 ]
[ 0.00832377  0.39746302 -0.9001654   0.9513525 ]
[-0.5148049 -0.5253884  0.8556755 -0.9570905]
[ 0.417919   -0.30732524  0.94678843 -0.20086375]
[-0.13593908  0.06800964  0.69921696  0.08020048]
[ 0.75887734 -0.86278063 -0.4913163  -0.31035393]
[-0.58619225  0.25302282 -0.48885062  0.8164327 ]
[ 0.00547632 -0.97077304  0.23667344 -0.40674514]
[-0.13365349 -0.78833574  0.08368549  0.86081743]
[ 0.21505022  0.6063552   0.56784266 -0.67635846]
[-0.52746904  0.9586462   0.11503773 -0.96331763]
[-0.37988076 -0.36280566  0.86582655 -0.8153599 ]
[ 0.3825068  -0.36736053  0.50912803 -0.8413812 ]
[-0.52373433 -0.4492433  -0.01713264  0.4928223 ]
[-0.567372   -0.76733774  0.5620124  -0.59583986]
[-0.88635933 -0.7631738  -0.12845741 -0.04295449]
[-0.40061656 -0.749377    0.06982562  0.41914576]
[-0.6495831   0.23568258 -0.57231086  0.77354544]
[ 0.

[ 0.52710724 -0.09658521  0.4422952   0.07787554]
[ 0.3987049  -0.7306122   0.19517587  0.75556296]
[ 0.3925113  -0.82193995  0.7906402   0.9359995 ]
[-0.389451    0.76926506  0.8965705  -0.15552244]
[-0.7269839   0.03348924 -0.7725289  -0.33400205]
[0.3416659  0.6783482  0.45935014 0.18407063]
[-0.17364568 -0.85959876  0.58572245 -0.08932294]
[ 0.78372574  0.5566704   0.02562244 -0.9191948 ]
[-0.20833984  0.12152426  0.11707384 -0.2046282 ]
[ 0.22301513 -0.4012456  -0.40786463  0.22348556]
[-0.41852766  0.46686345 -0.9519      0.9641547 ]
[ 0.2439337   0.07825533 -0.07789404 -0.05560871]
[-0.6757938   0.3778882  -0.4466419  -0.25882202]
[ 0.7823576  -0.6350664   0.73440194 -0.11537593]
[-0.60367095 -0.55443704  0.11202522  0.4610478 ]
[-0.4770768  -0.65088296  0.44922668 -0.6706731 ]
[ 0.08335508  0.5567889   0.87940854 -0.76710725]
[-0.92223525 -0.5278658  -0.518319   -0.56856537]
[-0.5858559  -0.85208863 -0.35888803 -0.40087608]
[-0.84330213  0.21467847  0.80876416  0.32462996]
[-0.

...

[ 0.33555976  0.10670654 -0.63844943 -0.7042055 ]
[0.71968365 0.37073484 0.9758041  0.9025978 ]
[0.04232433 0.4229557  0.16462542 0.6023922 ]
[-0.5187434  -0.37063107 -0.89499646  0.34356245]
[0.651518   0.33321065 0.34782344 0.96768045]
[ 0.9182158  -0.8094278  -0.5790545   0.34353772]
[ 0.16799246 -0.8869989  -0.40751585 -0.6289021 ]
[ 0.43746525 -0.9278886  -0.285691    0.04686312]
[ 0.3764026  -0.15127955  0.835115   -0.726475  ]
[ 0.10973959  0.6355063  -0.8962214  -0.56725305]
[-0.4547998   0.02251249  0.21646051 -0.11707844]
[-0.46113366  0.01988607  0.3947042   0.17349868]
[ 0.38356113 -0.3552148  -0.8391757  -0.9677594 ]
[ 0.28975812 -0.41148636  0.9029616  -0.861554  ]
[-0.19416872 -0.2281203  -0.06862366 -0.7962758 ]
[ 0.97749496 -0.72465444 -0.15724997  0.3135494 ]
[-0.7516874   0.8483626  -0.6797346  -0.12728725]
[0.3073667  0.08064894 0.7473594  0.07322491]
[ 0.28566742 -0.3646675  -0.58185947  0.93680197]
[ 0.9621856  -0.18489845 -0.412865    0.12160161]
[-0.6273418  -0.

[ 0.48755515  0.4237509   0.39513466 -0.66194016]
[ 0.3466428   0.7420465  -0.5343783   0.38960683]
[ 0.7771838  -0.01573148  0.3641647   0.8915528 ]
[ 0.19777034 -0.9181596  -0.63873374  0.25639126]
[-0.06537593  0.7650053   0.6967262  -0.6278864 ]
[ 0.85552615  0.76762843 -0.175105   -0.16111709]
[-0.22657368  0.567809   -0.41841245  0.468418  ]
[ 0.04808698 -0.37261286 -0.01781742 -0.38897994]
[-0.69447684 -0.93081164 -0.10250522  0.7166828 ]
[0.2773059 0.5315176 0.7032672 0.932372 ]
[ 0.22271526 -0.1108245  -0.65314996  0.5275727 ]
[ 0.28848052  0.58853877 -0.01841373  0.05821244]
[ 0.22226259 -0.830593    0.54086494 -0.50837725]
[-0.1280587   0.6133444  -0.6494398  -0.01588577]
[-0.5109466  -0.9131955  -0.21446535 -0.86833894]
[-0.69997525 -0.8736533  -0.9444645   0.32318243]
[-0.92045355  0.14742978  0.21994747 -0.14773346]
[-0.4419935  -0.59214306  0.1624577  -0.47022176]
[-0.1968886   0.27559686 -0.27347544 -0.4414125 ]
[ 0.7484368  -0.9266519  -0.9312452   0.35072318]
[0.72877

[-0.46495074 -0.8175608   0.9040543  -0.35468546]
[0.30147767 0.63591135 0.98569804 0.14719181]
[ 0.32410082 -0.32012302  0.72045374  0.7259425 ]
[ 0.731725  -0.9428394 -0.9269346  0.9211248]
[-0.9357525   0.48862112 -0.12638722  0.72995096]
[0.773958   0.5766138  0.0104243  0.00917274]
[-0.0551011  -0.23279706 -0.69991744 -0.0428935 ]
[ 0.16881329  0.08787688 -0.7950503   0.80553293]
[-0.52631956 -0.6027905  -0.01674866 -0.1688922 ]
[-0.36629894 -0.72621566 -0.79207075  0.14088586]
[-0.3486519   0.13970844 -0.7369385   0.93445563]
[-0.07857103 -0.6076994  -0.575593   -0.7492786 ]
[0.21292917 0.12974656 0.39529085 0.8687042 ]
[ 0.7123796  -0.77626115 -0.9345602  -0.52639943]
[-0.32491153  0.3239646   0.70135856 -0.9106488 ]
[-0.28862908 -0.15455641  0.78779304  0.12448039]
[ 0.7509086  -0.36186776  0.9608197  -0.90154976]
[ 0.9338654  -0.54548514 -0.35392907  0.8477309 ]
[ 0.48748684  0.68585896 -0.17569502  0.2222574 ]
[ 0.31406945 -0.7681595   0.04880345  0.76127076]
[-0.68770397 -0.

[-0.7407473   0.87484026  0.01266318 -0.6403701 ]
[-0.18156448  0.9541054  -0.71286976 -0.6287445 ]
[ 0.92543447  0.20224987 -0.86881876 -0.6578053 ]
[ 0.14185426  0.62556726 -0.7779267  -0.12365444]
[ 0.27199996  0.7268536   0.17333268 -0.93673253]
[ 0.22187836 -0.970192    0.05207731 -0.5761845 ]
[0.82554865 0.51785153 0.44777578 0.70561767]
[-0.48275506  0.9639734  -0.09524891  0.33183575]
[ 0.5100962   0.7219048  -0.80891514  0.7379942 ]
[-0.33081242 -0.24325626  0.8763832  -0.37947038]
[-0.3640863  -0.22813785 -0.31671336  0.02659929]
[ 0.6512594  -0.89904004 -0.10746711 -0.845703  ]
[-0.16934359 -0.9056482   0.52216053  0.58388644]
[-0.2428135   0.9643571   0.5076422  -0.20463702]
[ 0.97261655  0.9759396  -0.1444165   0.32986513]
[ 0.43537945  0.10780881 -0.00865751 -0.7413524 ]
[-0.13767204 -0.234542    0.2320736  -0.60018486]
[-0.8126914  0.7794244 -0.9198564  0.7672444]
[-0.5556458  -0.23050286  0.01061917 -0.29465005]
[-0.41158962 -0.06304653 -0.8944669  -0.53710014]
[ 0.2703

[-0.45594412  0.91559994 -0.4357918  -0.3463082 ]
[-0.35700232  0.71192706 -0.20518582 -0.8685396 ]
[-0.6961589 -0.306744   0.0254004  0.8004842]
[-0.3748284  0.5120893 -0.7491521 -0.6611481]
[-0.20413017  0.9277711  -0.79073983  0.48874494]
[-0.7757809   0.40621564  0.01669149  0.9810529 ]
[-0.96269757  0.74797416  0.17232639 -0.09300746]
[ 0.5641773   0.74851966 -0.8259244   0.25136882]
[-0.32468233  0.29466575 -0.96885556  0.5340349 ]
[-0.6197426  -0.7873671  -0.5790229  -0.75091505]
[-0.9913657  -0.5302017   0.12998842  0.7996532 ]
[ 0.12588759 -0.01750753 -0.28827956 -0.03428469]
[-0.6181841   0.93253607  0.02102241  0.814028  ]
[-0.9871569  -0.24233519  0.3833523  -0.11319912]
[-0.02972927 -0.4429629  -0.62868845  0.23674072]
[-0.45479873 -0.8731102   0.8995198   0.81413084]
[0.03000389 0.09390774 0.4763211  0.4699982 ]
[-0.9806165  -0.4605403  -0.05888671  0.5062101 ]
[-0.6461411  -0.26125532  0.7591795   0.9800434 ]
[-0.02160722 -0.16470626 -0.77965516 -0.6621853 ]
[ 0.601265  

In [ ]:
from scipy.ndimage.filters import uniform_filter1d
arr = uniform_filter1d(rewards, size=1)

In [ ]:
from matplotlib import pyplot as plt
x = []
y = []
for i,e in enumerate(arr):
    y.append(i)
    x.append(e)

In [ ]:
plt.plot(y, x)

In [ ]:
-np.prod(env.action_space.high.shape)